# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [80]:
# Do all imports and installs here
import pandas as pd
from pyspark.sql import SparkSession

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

#### 1. Importing Immigration dataset:
*Immigration dataset if quite large in size so its better to use **Apache Spark** to read the files in this case.*

In [79]:
%%time
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df_immig =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

CPU times: user 2.92 ms, sys: 0 ns, total: 2.92 ms
Wall time: 82.4 ms


In [71]:
#write to parquet
#df_spark.write.parquet("sas_data")
#df_spark=spark.read.parquet("sas_data")

In [85]:
df_immig.take(2)

[Row(cicid=6.0, i94yr=2016.0, i94mon=4.0, i94cit=692.0, i94res=692.0, i94port='XXX', arrdate=20573.0, i94mode=None, i94addr=None, depdate=None, i94bir=37.0, i94visa=2.0, count=1.0, dtadfile=None, visapost=None, occup=None, entdepa='T', entdepd=None, entdepu='U', matflag=None, biryear=1979.0, dtaddto='10282016', gender=None, insnum=None, airline=None, admnum=1897628485.0, fltno=None, visatype='B2'),
 Row(cicid=7.0, i94yr=2016.0, i94mon=4.0, i94cit=254.0, i94res=276.0, i94port='ATL', arrdate=20551.0, i94mode=1.0, i94addr='AL', depdate=None, i94bir=25.0, i94visa=3.0, count=1.0, dtadfile='20130811', visapost='SEO', occup=None, entdepa='G', entdepd=None, entdepu='Y', matflag=None, biryear=1991.0, dtaddto='D/S', gender='M', insnum=None, airline=None, admnum=3736796330.0, fltno='00296', visatype='F1')]

In [86]:
df_immig.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

#### 2. Importing world_temperature dataset:
*World temperature dataset if quite large in size so its better to use **Apache Spark** to read the files in this case.*

In [73]:
# %%time
# Read in the data here
# df_temp=pd.read_csv('../../data2/GlobalLandTemperaturesByCity.csv')

In [74]:
%%time
df_temp=spark.read.format('csv').option('header','True').load('../../data2/GlobalLandTemperaturesByCity.csv')

CPU times: user 1.89 ms, sys: 353 µs, total: 2.24 ms
Wall time: 352 ms


In [75]:
df_temp.first()

Row(dt='1743-11-01', AverageTemperature='6.068', AverageTemperatureUncertainty='1.7369999999999999', City='Århus', Country='Denmark', Latitude='57.05N', Longitude='10.33E')

#### 3. Importing airport_code dataset:
*airport_code dataset is not that much large in size its better to use **Pandas** to read the files in this case.*

In [82]:
# Read in the data here
df_port=pd.read_csv('airport-codes_csv.csv')

In [83]:
df_port.head(2)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"


#### 4. Importing us_cities_demographics dataset:
*us_cities_demographics dataset is not that much large in size its better to use **Pandas** to read the files in this case.*

In [81]:
# Read in the data here
df_demog=pd.read_csv('us-cities-demographics.csv',delimiter=';')


In [59]:
df_demog.head(2)

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723


### Step 2: Explore and Assess the Data




### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### 1. Exploring Immigration dataset:

In [11]:
# Checking for missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 29 columns):
Unnamed: 0    1000 non-null int64
cicid         1000 non-null float64
i94yr         1000 non-null float64
i94mon        1000 non-null float64
i94cit        1000 non-null float64
i94res        1000 non-null float64
i94port       1000 non-null object
arrdate       1000 non-null float64
i94mode       1000 non-null float64
i94addr       941 non-null object
depdate       951 non-null float64
i94bir        1000 non-null float64
i94visa       1000 non-null float64
count         1000 non-null float64
dtadfile      1000 non-null int64
visapost      382 non-null object
occup         4 non-null object
entdepa       1000 non-null object
entdepd       954 non-null object
entdepu       0 non-null float64
matflag       954 non-null object
biryear       1000 non-null float64
dtaddto       1000 non-null object
gender        859 non-null object
insnum        35 non-null float64
airline       967 non

In [12]:
df.describe()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,arrdate,i94mode,depdate,i94bir,i94visa,count,dtadfile,entdepu,biryear,insnum,admnum
count,1.000000e+03,1.000000e+03,1000.0,1000.0,1000.000000,1000.00000,1000.000000,1000.000000,951.000000,1000.000000,1000.000000,1000.0,1.000000e+03,0.0,1000.000000,35.000000,1.000000e+03
mean,1.542097e+06,3.040461e+06,2016.0,4.0,302.928000,298.26200,20559.680000,1.078000,20575.037855,42.382000,1.859000,1.0,2.016042e+07,NaN,1973.618000,3826.857143,6.937237e+10
std,9.152879e+05,1.799818e+06,0.0,0.0,206.485285,202.12039,8.995027,0.485955,24.211234,17.903424,0.386353,0.0,4.951657e+01,NaN,17.903424,221.742583,2.338134e+10
min,1.092500e+04,1.320800e+04,2016.0,4.0,103.000000,103.00000,20545.000000,1.000000,20547.000000,1.000000,1.000000,1.0,2.016040e+07,NaN,1923.000000,3468.000000,0.000000e+00
25%,7.214422e+05,1.412170e+06,2016.0,4.0,135.000000,131.00000,20552.000000,1.000000,20561.000000,30.750000,2.000000,1.0,2.016041e+07,NaN,1961.000000,3668.000000,5.599301e+10
50%,1.494568e+06,2.941176e+06,2016.0,4.0,213.000000,213.00000,20560.000000,1.000000,20570.000000,42.000000,2.000000,1.0,2.016042e+07,NaN,1974.000000,3887.000000,5.931477e+10
75%,2.360901e+06,4.694151e+06,2016.0,4.0,438.000000,438.00000,20567.250000,1.000000,20580.000000,55.000000,2.000000,1.0,2.016042e+07,NaN,1985.250000,3943.000000,9.343623e+10
max,3.095749e+06,6.061994e+06,2016.0,4.0,746.000000,696.00000,20574.000000,9.000000,20715.000000,93.000000,3.000000,1.0,2.016080e+07,NaN,2015.000000,4686.000000,9.502151e+10


In [14]:
# Drop duplicates value
#df.drop_duplicates()

#### 2. Exploring world_temperature dataset:

In [87]:
#Shape of the dataset
df_temp.printSchema()

root
 |-- dt: string (nullable = true)
 |-- AverageTemperature: string (nullable = true)
 |-- AverageTemperatureUncertainty: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)



In [88]:
# Checking missing values
df_temp.describe()

DataFrame[summary: string, dt: string, AverageTemperature: string, AverageTemperatureUncertainty: string, City: string, Country: string, Latitude: string, Longitude: string]

In [17]:
# Drop Duplicates Values
df_port.drop_duplicates(inplace=True)

#### 3. Exploring airport_code dataset:

In [20]:
#Shape of the dataset
df_port.shape

(55075, 12)

In [16]:
# Checking missing values
df_port.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55075 entries, 0 to 55074
Data columns (total 12 columns):
ident           55075 non-null object
type            55075 non-null object
name            55075 non-null object
elevation_ft    48069 non-null float64
continent       27356 non-null object
iso_country     54828 non-null object
iso_region      55075 non-null object
municipality    49399 non-null object
gps_code        41030 non-null object
iata_code       9189 non-null object
local_code      28686 non-null object
coordinates     55075 non-null object
dtypes: float64(1), object(11)
memory usage: 5.0+ MB


In [17]:
# Drop Duplicates Values
df_port.drop_duplicates(inplace=True)

In [18]:
df_port.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55075 entries, 0 to 55074
Data columns (total 12 columns):
ident           55075 non-null object
type            55075 non-null object
name            55075 non-null object
elevation_ft    48069 non-null float64
continent       27356 non-null object
iso_country     54828 non-null object
iso_region      55075 non-null object
municipality    49399 non-null object
gps_code        41030 non-null object
iata_code       9189 non-null object
local_code      28686 non-null object
coordinates     55075 non-null object
dtypes: float64(1), object(11)
memory usage: 5.5+ MB


#### 3. Exploring us_cities_demographics dataset:

In [30]:
# Shape of the dataset
df_demog.shape

(2891, 12)

In [31]:
# Checking for missing values
df_demog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 12 columns):
City                      2891 non-null object
State                     2891 non-null object
Median Age                2891 non-null float64
Male Population           2888 non-null float64
Female Population         2888 non-null float64
Total Population          2891 non-null int64
Number of Veterans        2878 non-null float64
Foreign-born              2878 non-null float64
Average Household Size    2875 non-null float64
State Code                2891 non-null object
Race                      2891 non-null object
Count                     2891 non-null int64
dtypes: float64(6), int64(2), object(4)
memory usage: 271.1+ KB


In [33]:
# Dropping duplicates
df_demog.drop_duplicates(inplace=True)

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

#### Using Star Schema:
##### Dimension Tables:
> 1. Immigrant Table
    * Columns: gender, biryear, occup, i94bir
##### Fact Table:

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

### 4.3 Data dictionary 
_Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file._

#### Immigration Data:
~~~
  * cicid    -  
  * i94yr    - 4 digit year
  * i94mon   - Numeric month
  * i94cit   - This format shows all the valid and invalid codes for processing
  * i94res   - This format shows all the valid and invalid codes for processing
  * i94port  - This format shows all the valid and invalid codes for processing
  * arrdate  - is the Arrival Date in the USA. It is a SAS date numeric field that a 
               permament format has not been applied.  Please apply whichever date format 
               works for you.
  * i94mode  - There are missing values as well as not reported (9)
               	1 = 'Air'
                2 = 'Sea'
                3 = 'Land'
                9 = 'Not reported' ;   
  * i94addr  - There is lots of invalid codes in this variable and the list below 
               shows what we have found to be valid, everything else goes into 'other'
  * depdate  - is the Departure Date from the USA. It is a SAS date numeric field that 
               a permament format has not been applied.  Please apply whichever date format 
               works for you.
  * i94bir   - Age of Respondent in Years
  * i94visa  - Visa codes collapsed into three categories:
               1. 1 = Business
               2. 2 = Pleasure
               3. 3 = Student
  * count    - Used for summary statistics
  * dtadfile - Character Date Field - Date added to I-94 Files
  * visapost - Department of State where where Visa was issued
  * occup    - Occupation that will be performed in U.S.
  * entdepa  - Arrival Flag - admitted or paroled into the U.S.
  * entdepd  - Departure Flag - Departed, lost I-94 or is deceased
  * entdepu  - Update Flag - Either apprehended, overstayed, adjusted to perm residence
  * matflag  - Match flag - Match of arrival and departure records
  * biryear  - 4 digit year of birth
  * dtaddto  - Character Date Field - Date to which admitted to U.S. (allowed to stay until)
  * gender   - Non-immigrant sex
  * insnum   - INS number
  * airline  - Airline used to arrive in U.S.
  * admnum   - Admission Number
  * fltno    - Flight number of Airline used to arrive in U.S.
  * visatype - Class of admission legally admitting the non-immigrant to temporarily stay in U.S.
  
 ~~~

#### World Temperature Data:
```
* dt
* AverageTemperature
* AverageTemperatureUncertainty
* City
* Country
* Latitude
* Longitude
```

#### U.S. City Demographic Data:
```
* City
* State
* Median Age
* Male Population
* Female Population
* Total Population
* Number of Veterans
* Foreign-born
* Average Household Size
* State Code
* Race
* Count
```

#### Airport Codes:
```
* ident  
* type 
* name 
* elevation_ft 
* continent
* iso_country
* iso_region
* municipality
* gps_code
* iata_code
* local_code
* coordinates
```       

### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.